## Korrektur 

die Aufgabe f) funktioniert bei mir leider nicht. -3

12

In [2]:
# a)
import numpy as np
def lu_decomp(A):
    n = len(A)
    p = list(range(n))
    for k in range(n - 1):
        # finde maximales Element auf Spalte k
        i = k
        for j in range(k + 1, n):
            if A[j][k] > A[k][k]:
                i = j
        if i != k: # swap rows k and i
            p[k], p[i] = p[i], p[k] # swap using "tuple unpacking"
            A[[k,i],:] = A[[i,k],:]
        # Elimininationsschritt
        for t in range(k + 1, n):
            A[t][k] /= A[k][k]
            for s in range(k + 1, n):
                A[t][s] -= A[k][s] * A[t][k]
    return p

# test
A = np.array([[3., 1., 6.], [2., 1., 3.], [1., 1., 1.]])
A_orig = A.copy()
p = lu_decomp(A)
print(p)
print(A)

[0, 2, 1]
[[ 3.          1.          6.        ]
 [ 0.33333333  0.66666667 -1.        ]
 [ 0.66666667  0.5        -0.5       ]]


In [3]:
# b)
def lu_solve(p, A, b):
    c = b.transpose()
    if len(b) == 1:
        n = len(c)
        m = 1
    else:
        n, m = c.shape
    x = np.zeros([n, m])
    y = np.zeros([n, m])
    # Vorwörtseinsetzen Ly = Pb
    for i in range(n):
        y[i] = c[p[i]]
        for j in range(i):
            y[i] -= A[i][j] * y[j]
    # Rückwärtseinsetzen Rx = y
    for l in range(n - 1, -1, -1):
        x[l] = y[l]
        for q in range(l + 1, n):
            x[l] -= x[q] * A[l][q]
        x[l] /= A[l][l]
    # reshape if m == 1
    if m == 1:
        x = np.array(x.reshape(n, 1))
    return x.transpose()

# test
b = np.array([[2, 7, 4]])
x = lu_solve(p, A, b)
print(x)

[[ 19.  -7.  -8.]]


In [4]:
# d)
n = 40
T = np.random.rand(n, n)
print(T)
T_orig = T.copy()
bT = np.eye(n) # Einheitsmatrix
pT = lu_decomp(T)
InvT = lu_solve(pT, T, bT)
print(InvT)
print(InvT.dot(T_orig.transpose())) # test ob das Produkt zweier Matrizen eine Einheitsmatrix ist
norm = np.linalg.norm(T_orig.dot(InvT.transpose()) - bT, np.inf)
print(norm)

[[ 0.55080712  0.0252388   0.01858274 ...,  0.25724593  0.35198997
   0.79340511]
 [ 0.84080534  0.33666257  0.53859686 ...,  0.0356134   0.72933223
   0.12931366]
 [ 0.13068085  0.94752191  0.76567964 ...,  0.13379294  0.16238563
   0.88365469]
 ..., 
 [ 0.24936089  0.15786513  0.51015251 ...,  0.93881846  0.89459819
   0.71907259]
 [ 0.22131735  0.05972775  0.21322484 ...,  0.42699447  0.73825829
   0.39348838]
 [ 0.80472699  0.72091407  0.13146348 ...,  0.86739473  0.10382197
   0.54295915]]
[[  4.18933334e-02  -1.27838866e+00  -1.56512449e-02 ...,  -2.27833494e-01
   -4.00678859e-02   1.60813636e-01]
 [  7.90679968e-01   5.88236327e-02  -3.41623886e-01 ...,  -3.67445700e-01
   -1.06339277e-01   2.18328474e-01]
 [ -1.38314362e+00   6.12549114e-01   9.67484398e-01 ...,  -6.01912257e-01
    1.14345971e-01  -4.24783899e-01]
 ..., 
 [  3.06200081e-01   5.42615101e-01  -3.85699069e-01 ...,   7.83872327e-01
   -1.39523315e-02   2.03657737e-01]
 [ -7.69653713e-01  -1.55351777e+00   5.60156

In [5]:
# e)
def cond_inf(A):
    A_orig = A.copy()
    n = len(A)
    b = np.eye(n)
    p = lu_decomp(A)
    InvA = lu_solve(p, A, b)
    # finde das maximale Zeilensumme
    maxA = sum(A_orig[0])
    maxInvA = sum(InvA[0])
    for i in range(n):
        if sum(A_orig[i]) > maxA:
            maxA = sum(A_orig[i])
        if sum(InvA[i]) > maxInvA:
            maxInvA = sum(InvA[i])
    return maxA * maxInvA

# test
print(cond_inf(T_orig))

17.1757114896


In [6]:
import math
x = 6
N = np.arange(0, x, 2)
# Konditionszahl
kz1 = np.zeros(x / 2 + 1)
kz1[0] = 1
print(kz1)
kz2 = np.zeros(x / 2 + 1)
for n in N:
    # Monombasis
    def Mphy(k, x):
        return x**k
    def Mpkt(i):
        return -1 + 2 * i / (n - 1)
    V1 = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            V1[i][j] = Mphy(j, Mpkt(i))
    print(V1)
    #kz1[n/2] = cond_inf(V1)
    # Tschebytschow-Basis
    def Tphy(k, x):
        return math.cos(k * math.acos(x))
    def Tpkt(i):
        return cos(i * math.pi / n)
    V2 = np.zeros([n, n])
    for k in range(n):
        for l in range(n):
            V2[k][l] = Mphy(l, Mpkt(k))
    #kz2[n/2] = cond_inf(V2)
    
import matplotlib.pyplot as plt
plt.plot(N, kz1)
plt.plot(N, kz2)
plt.show()

/home/elias/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/elias/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[ 1.  0.  0.  0.]
[]
[[ 1. -1.]
 [ 1.  1.]]
[[ 1.         -1.          1.         -1.        ]
 [ 1.         -0.33333333  0.11111111 -0.03703704]
 [ 1.          0.33333333  0.11111111  0.03703704]
 [ 1.          1.          1.          1.        ]]


ValueError: x and y must have same first dimension